In [100]:
import pandas as pd
import numpy as np

import ingestion as ing

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

In [41]:
# read the users
users_ddf = ing.read_parquet(
    path=ing.build_path(folderind="ba", filename="users_with_scores", ext=".parquet", basepath=ing.REFINED_PATH),
    keepcols=None,
    assume_missing=False,
    mode="lazy")

In [42]:
users_df = users_ddf.compute()

In [45]:
users_df.columns

Index(['n_ratings', 'n_reviews', 'uid', 'username', 'joined', 'location',
       'country', 'cfm_score', 'exp_score', 'xpl_score', 'adv_score', 'is_cfm',
       'is_exp', 'is_xpl', 'is_adv'],
      dtype='object')

Time-Analysis of BeerAdvocate.com Attractiveness for Classes

In [112]:
class_indicators = ["is_cfm", "is_exp", "is_xpl", "is_adv"]

# number of classified users
len(users_df.query("+".join(class_indicators) + " != 0"))

41395

In [121]:
by_joined_y = users_df.groupby(by=users_df.joined.dt.year, as_index=False)

In [98]:
# plot the time-evolution of the size of classes


        is_cfm  is_exp  is_xpl  is_adv
108          0       0       1       1
145          0       0       1       1
151          0       0       1       1
153          0       0       1       1
161          0       0       1       1
...        ...     ...     ...     ...
124329       0       0       0       0
135711       0       0       0       0
137794       0       0       1       0
144087       0       0       0       0
153678       0       0       0       0

[78 rows x 4 columns]
